In [1]:
import config
import sqlalchemy
import functions
import statsapi as mlb

In [2]:
gamePks = functions.read_gamePks()

In [3]:
test_pk = gamePks['2019'][100]

In [4]:
_sql_alchemy_connection = (
                                f'mysql+mysqlconnector://'
                                f'{config.user}:{config.password}'
                                f'@{config.host}:{config.port}'
                                f'/{config.schema}'
                           )

db = sqlalchemy.create_engine(_sql_alchemy_connection,
                              echo = False,
                              connect_args = {'ssl_disabled' : True})

In [5]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

from sqlalchemy import Column, Integer, String,DateTime,Date,Time
# Declare mapping for the game table 
class Game(Base):
    __tablename__ = 'games'
    
    pk = Column(Integer)
    type = Column(String(1))
    doubleHeader = Column(String(1))
    id = Column(String(150), primary_key=True)
    gamedayType = Column(String(1))
    tiebreaker = Column(String(1))
    gameNumber = Column(Integer)
    calenderEventId = Column(String(50))
    season = Column(Integer)
    
    dateTime = Column(DateTime)
    originalDate = Column(Date)
    dayNight = Column(String(12))
    time = Column(Time)
    
    abstractGameState = Column(String(12))
    codedGameState = Column(String(3))
    detailedState = Column(String(12))
    statusCode = Column(String(3))
    abstractGameCode = Column(String(3))
    
    homeTeam_id = Column(Integer)
    awayTeam_id = Column(Integer)
    
    condition = Column(String(25))
    temp = Column(Integer)
    wind = Column(String(50))
    
    venue_id = Column(Integer)
    
    home_probablePitcher = Column(Integer)
    away_probablePitcher = Column(Integer)

In [6]:
# Create the table 
Base.metadata.create_all(db)

In [7]:
# Create instance of the mapped class 
def create_gameRecord(pk):
    from datetime import datetime
    import time
    api_call = mlb.get('game',{'gamePk':pk})
    gameData = api_call['gameData']
    game = gameData['game']
    _datetime = gameData['datetime']
    status = gameData['status']
    weather = gameData['weather']
    probablePitchers = gameData['probablePitchers']
    
    record = Game(pk=game['pk'],
                 type=game['type'],
                 doubleHeader=game['doubleHeader'],
                 id=game['id'],
                 gamedayType=game['gamedayType'],
                 tiebreaker=game['tiebreaker'],
                 gameNumber=game['gameNumber'],
                 calenderEventId=game['calendarEventID'],
                 season=game['season'],
                  
                 dateTime=datetime.strptime(_datetime['dateTime'],'%Y-%m-%dT%H:%M:%SZ'),
                 originalDate=datetime.strptime(_datetime['originalDate'],"%Y-%m-%d"),
                 dayNight=_datetime['dayNight'],
                 time=datetime.strptime(_datetime['time']+_datetime['ampm'],"%H:%M%p"),
                  
                 abstractGameState=status['abstractGameState'],
                 codedGameState=status['codedGameState'],
                 detailedState=status['detailedState'],
                 statusCode=status['statusCode'],
                 abstractGameCode=status['abstractGameCode'],
                  
                 homeTeam_id=gameData['teams']['home']['id'],
                 awayTeam_id=gameData['teams']['away']['id'],
                  
                 condition=weather.get('condition','null'),
                 temp=weather.get('temp','null'),
                 wind=weather.get('wind','null'),
                  
                 venue_id=gameData['venue']['id'],
                  
                 home_probablePitcher=probablePitchers.get('home',{'null':'null'}).get('id','null'),
                 away_probablePitcher=probablePitchers.get('away',{'null':'null'}).get('id','null') 
                 )
    return record

In [8]:
test_record = create_gameRecord(test_pk)

In [9]:
# Create a Session 
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=db)
session = Session()


In [10]:
# Adding and updating objects
session.add(test_record)

In [11]:
# instance is not added until we 'flush' with a query
test_query = session.query(Game)
test_query

In [12]:
session.commit()

In [13]:
db.execute("select * from games").fetchall()

[(566651, 'S', 'N', '2019/02/28/minmlb-tbamlb-1', 'Y', 'N', 1, '14-566651-2019-02-28', 2019, datetime.datetime(2019, 2, 28, 23, 35), datetime.date(2019, 2, 28), 'night', datetime.timedelta(seconds=23700), 'Final', 'F', 'Final', 'F', 'F', 139, 142, 'Clear', 74, '10 mph, L To R', 2534, 605483, 606167)]

In [ ]:
session.add_all([create_gameRecord(x) for x in gamePks['2019']])

In [ ]:
session.commit()

In [ ]:
db.execute("select * from games").fetchall()

In [ ]:
db.table_names()

In [ ]:
db.execute("drop table games")